In [9]:
import numpy as np
import time
from OF import OptimumFilter  # Replace with actual module path
from OF_trigger import OFtrigger


sampling_frequency = 3906250

data = np.load("/ceph/dwong/delight/Ka_traces_1.npz")

template = np.load("../templates/template_K_alpha_tight.npy")
noise_psd = np.load("../templates/noise_psd_from_MMC.npy")
loaded_traces = data['data']
data.close()

# Ensure correct shape

assert len(template) == loaded_traces.shape[1], "Trace length must match template length"

# Benchmark function using fit_with_shift
def benchmark_filter_with_shift(FilterClass, template, psd, sampling_frequency, traces):
    of = FilterClass(template, psd, sampling_frequency)
    start = time.time()
    results = []
    for i in range(traces.shape[0]):
        results.append(of.fit(traces[i]))
    elapsed = time.time() - start
    return elapsed, results

# Run benchmarks
t_old, results_old = benchmark_filter_with_shift(OptimumFilter, template, noise_psd, sampling_frequency, loaded_traces)
t_new, results_new = benchmark_filter_with_shift(OFtrigger, template, noise_psd, sampling_frequency, loaded_traces)
print(f"OptimumFilter time (fit_with_shift): {t_old:.4f}s")
print(f"OFtrigger time (fit_with_shift): {t_new:.4f}s")
print(f"Speedup: {t_old / t_new:.2f}x")

OptimumFilter time (fit_with_shift): 4.2987s
OFtrigger time (fit_with_shift): 2.0985s
Speedup: 2.05x


In [5]:
results_old[10:20] 

[(10414.42191904015, 1.030801645592369),
 (10405.665579890223, 0.9785365274571997),
 (10416.386487652877, 1.0108269016491889),
 (10411.39540220257, 0.9890808943602902),
 (10395.465460254754, 1.0254867881971248),
 (10414.140305288478, 0.9932377290421555),
 (10411.829155000516, 1.032540640745023),
 (10416.330759369554, 1.0021761522403765),
 (10415.423674732101, 1.0033656299552631),
 (10416.520771292313, 1.0048662332395768)]

In [6]:
results_new[10:20]

[(10414.421090248574, 0.2577644309104817),
 (10405.6662550171, 0.24467661264522758),
 (10416.386732229108, 0.2527123004807928),
 (10411.395212886395, 0.24727356397963912),
 (10395.465671545377, 0.2563758578990752),
 (10414.14057650647, 0.24831628806896788),
 (10411.829917958335, 0.2581894130602971),
 (10416.331222260222, 0.2505640080841584),
 (10415.422702388238, 0.25092952484588427),
 (10416.520472719629, 0.2512248667950818)]

In [8]:
len(results_new)

2182